In [143]:
import pandas as pd
from datetime import datetime, date
import asyncio
import aiohttp
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np

import seaborn as sns
import matplotlib
from sklearn.ensemble import IsolationForest
nest_asyncio.apply()

In [302]:
dateparser = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M")
df = pd.read_csv("DataSet.csv", parse_dates = ['Datetime'], date_parser=dateparser)

In [304]:
df.to_csv('events_db.csv')

In [145]:
import sklearn

In [146]:
df.groupby(by=df['Datetime'].dt.date).count().sort_values('Category')

,Datetime,Category,AddressEac,BuildingEac,Lat,Lon,District
Datetime,,,,,,,
2020-05-17,101,101,40,89,101,101,101
2020-04-26,103,103,31,96,103,103,103
2019-05-11,104,104,42,88,104,104,104
2020-05-03,105,105,38,86,105,105,105
2020-05-02,105,105,41,95,105,105,105
...,...,...,...,...,...,...,...
2019-09-09,765,765,339,737,765,765,765
2019-08-06,989,989,446,966,989,989,989
2020-01-12,1062,1062,51,1042,1062,1062,1062


In [147]:
df.loc['2019-07-02':'2019-07-02'].to_csv('2019-07-02.csv', encoding='windows-1251')

In [148]:
df.loc['2019-07-02':'2019-07-02'].groupby(['Category']).Category.agg('count').to_frame('count').reset_index()


,Category,count


In [149]:
df.Category.unique()

array(['Слабый напор или отсутствие горячего водоснабжения',
       'Плохое качество горячего водоснабжения',
       'Слабый напор или отсутствие холодного водоснабжения',
       'Обращения с жалобой', 'ДТП с пострадавшими людьми', 'Пожары',
       'Горение мусора, листьев'], dtype=object)

In [150]:
categories = [
    'Плохое качество горячего водоснабжения',
    'Слабый напор или отсутствие холодного водоснабжения',
    'Слабый напор или отсутствие горячего водоснабжения',
    'Обращения с жалобой',
    'ДТП с пострадавшими людьми',
    'Пожары',
    'Горение мусора, листьев'
]

In [151]:
districts = [
    'Центральный', 
    'Красногвардейский', 
    'Петродворцовый', 
    'Красносельский', 
    'Колпинский', 
    'Выборгский',
    'Пушкинский',
    'Калининский',
    'Невский',
    'Фрунзенский',
    'Василеостровский',
    'Московский',
    'Приморский',
    'Петроградский',
    'Кировский',
    'Адмиралтейский',
    'Курортный',
    'Кронштадтский',
]

In [152]:
async def async_request(url, params):
    async with aiohttp.request(method='GET', url=url, params=params) as response:
        result = await response.json()
    return result


In [153]:
df = df[df['District'].isin(districts)]

In [154]:
df['District'] = df['District'].replace(districts, list(range(len(districts))))

In [155]:
df['Category'] = df['Category'].replace(categories, list(range(len(categories))))

In [156]:
df_filtered = df[['Datetime','Category', 'Lat', 'Lon', 'District']]
df_filtered['date'] = date(2019,1,1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [169]:
df_filtered['date'] = pd.to_datetime(
    df_filtered['Datetime'], format='%Y-%m-%d %H'
).dt.date
df_filtered['hour'] = pd.to_datetime(
    df_filtered['Datetime'], format='%Y-%m-%d %H'
).dt.hour

In [323]:
dateparser = lambda x: datetime.strptime(x, "%d.%m.%Y %H:%M")
weather = pd.read_csv('26063.01.01.2019.12.08.2020.1.0.0.ru.utf8.00000000.csv', sep=';', parse_dates = ['Datetime'], date_parser=dateparser)

In [324]:
weather = weather.groupby(weather['Datetime'].dt.date).T.agg('mean').to_frame('mean').reset_index()
weather

,Datetime,mean
0,2019-01-01,-0.9750
1,2019-01-02,0.6250
2,2019-01-03,-2.7375
3,2019-01-04,-5.9125
4,2019-01-05,-4.9250
...,...,...
585,2020-08-08,20.0500
586,2020-08-09,21.2125
587,2020-08-10,20.3750
588,2020-08-11,14.4500


In [245]:
date_hour_count = df_filtered.groupby(
    ['date', 'hour', 'Category', 'District']
).Category.agg('count').to_frame('count').reset_index()
date_hour_count
date_hour_count['weather'] = [
    weather[
        weather['Datetime'] == row['date']
    ]['mean'].tolist()[0]
    for index, row in date_hour_count.iterrows()
]

In [299]:
date_hour_count['dayoff'] = [
    dayoffs_dict[str(row['date'])]
    for index, row in date_hour_count.iterrows()
]

In [ ]:
df_filtered['count'] = [
    date_hour_count[
        (date_hour_count['date'] == row['date']) 
        & (date_hour_count['hour'] == row['hour'])
    ]['count'].tolist()[0] for index, row in df_filtered.iterrows()
]

In [319]:
df_filtered['dayoff'] = [
    dayoffs_dict[str(row['Datetime'].date())]
    for index, row in df_filtered.iterrows()
]

In [278]:
dateparser = lambda x: datetime.strptime(x, "%m/%d/%Y")
dayoffs = pd.read_csv("dayoffs.csv", parse_dates = ['Date'], date_parser=dateparser)

In [269]:
dayoffs[dayoffs['Date'] == date_hour_count['date']]

In [329]:
df_filtered['weather'] = [
    weather[weather['Datetime'] == row['Datetime'].date()]['mean'].tolist()[0]
    for index, row in df_filtered.iterrows()
]

In [296]:
dayoffs_dict = dict(zip(dayoffs.Date.dt.strftime('%Y-%m-%d'), dayoffs.dayoff))


In [301]:
date_hour_count.to_csv('dth.csv')

In [335]:
df_filtered.sort_values(['date','hour'])[:14]

,Datetime,Category,Lat,Lon,District,date,hour,dayoff,weather
0,2019-01-01 00:08:00,2,59.945053,30.341370,0,2019-01-01,0,1,-0.975
1,2019-01-01 00:30:00,2,59.955637,30.420857,1,2019-01-01,0,1,-0.975
2,2019-01-01 00:41:00,2,59.918879,30.417112,1,2019-01-01,0,1,-0.975
115653,2019-01-01 00:06:00,5,60.026792,30.327074,5,2019-01-01,0,1,-0.975
115654,2019-01-01 00:07:00,5,60.054293,30.343064,5,2019-01-01,0,1,-0.975
115655,2019-01-01 00:07:00,5,60.054293,30.343064,5,2019-01-01,0,1,-0.975
115656,2019-01-01 00:07:00,5,60.053820,30.350525,5,2019-01-01,0,1,-0.975
115657,2019-01-01 00:08:00,5,60.054293,30.343064,5,2019-01-01,0,1,-0.975
115658,2019-01-01 00:08:00,5,60.054293,30.343064,5,2019-01-01,0,1,-0.975
115659,2019-01-01 00:09:00,5,60.056400,30.342000,5,2019-01-01,0,1,-0.975


In [331]:
df_filtered.to_csv('events.csv')

In [325]:
weather

,Datetime,mean
0,2019-01-01,-0.9750
1,2019-01-02,0.6250
2,2019-01-03,-2.7375
3,2019-01-04,-5.9125
4,2019-01-05,-4.9250
...,...,...
585,2020-08-08,20.0500
586,2020-08-09,21.2125
587,2020-08-10,20.3750
588,2020-08-11,14.4500
